#### Stage 3
# Origin Destination Data - Temporal Overview
with matplotlib

#### Generating Temporal Plots with OD Data
This notebook contains the code to generate **radial and horizontal bar charts** similarly to those at [stage one](https://github.com/klavere/mCLOUDxMNDvis/blob/main/1_flowData_temporalOverview.ipynb) but based on the origin-destination dataset [cycleAndFlowODdf.csv](https://www.mcloud.de/downloads/ingrid-group_ige-iplug-mcloud/9A101FEC-3502-495A-9D93-BD8329A9D8AC/cycleAndFlowODdf.csv).
Again, two separate functions are written to plot the circular or the horizontal version and a third to filter and aggregate the data and chose which version to plot.

#### Versions of the used packages:
- pandas: 0.24.2
- numpy: 1.16.4
- matplotlib: 3.2.1
- ipywidgets: 7.5.1

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from math import pi
import numpy as np
import ipywidgets as pyw

In [3]:
def csvtodf_SC(path):
    data = pd.read_csv('data/'+path+'.csv',
                       delimiter=';',
                       skipinitialspace=True,
                       skiprows=0)
    df = pd.DataFrame(data)
    return df;

def csvtodf_C(path):
    data = pd.read_csv('data/'+path+'.csv',
                       delimiter=',',
                       skipinitialspace=True,
                       skiprows=0)
    df = pd.DataFrame(data)
    return df;

## load data
processed in 24.0

In [4]:
ODdf = csvtodf_SC('cycleAndFlowODdf')
ODdf.head()

,startCellID,endCellID,ODCoords,dayInt,hour,weekday,dayType,Bus,Rail,Tram,...,traveltime_mean_Rail,traveltime_mean_Tram,traveltime_mean_UBahn,traveltime_mean_privat,public,moves,traveltime_mean_public,dist_mean_public,traveltime_mean_moves,dist_mean_moves
0,1,1,"['[48.15396752884293, 11.51032914199828]', '[4...",0,0,Monday,MonThu,2.0,0.0,0.0,...,0.0,0.0,0.0,656.071429,2.0,16.0,251.25,0.06575,453.660714,0.066696
1,1,1,"['[48.15396752884293, 11.51032914199828]', '[4...",0,1,Monday,MonThu,0.0,0.0,0.0,...,0.0,0.0,0.0,556.000000,0.0,8.0,0.00,0.00000,278.000000,0.058250
2,1,1,"['[48.15396752884293, 11.51032914199828]', '[4...",0,2,Monday,MonThu,0.0,0.0,0.0,...,0.0,0.0,0.0,440.225806,0.0,31.0,0.00,0.00000,220.112903,0.064032
3,1,1,"['[48.15396752884293, 11.51032914199828]', '[4...",0,3,Monday,MonThu,0.0,0.0,0.0,...,0.0,0.0,0.0,594.583333,0.0,12.0,0.00,0.00000,297.291667,0.138208
4,1,1,"['[48.15396752884293, 11.51032914199828]', '[4...",0,4,Monday,MonThu,0.0,0.0,0.0,...,0.0,0.0,0.0,598.962963,0.0,27.0,0.00,0.00000,299.481481,0.096148


## constants for plotting function

In [5]:
days = ['', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dayTypes = ['', 'MonThu', 'Fri', 'Weekend']

modes = ['moves', 'privat', 'public', 'Rail', 'UBahn', 'Tram', 'Bus']
modesPlt = ['', 'privat', 'public']

traveltime_mean_modes = ['']*7
for i in range(0,7):
    traveltime_mean_modes[i] = 'traveltime_mean_'+modes[i]

dist_mean_modes = ['']*7
for i in range(0,7):
    dist_mean_modes[i] = 'dist_mean_'+modes[i]
    
distMax = []
for k in dist_mean_modes:
    distMax.append(max(ODdf[k]))
maxDist = max(distMax)

modesColor = {'privat':'#BFBFBF',#'#999999',
              'Rail':'#4daf4a',
              'UBahn':'#377eb8',
              'Tram':'#e41a1c',
              'Bus':'#984ea3',
              'public':'#ff7f00', # '#ff7f00',
              'moves':'#a65628'} # '#FFFFFF'}
# including ColorBrewer Colors from: https://colorbrewer2.org/#type=qualitative&scheme=Set1&n=9

d = {}
modesAggMean = dict((key, ['mean']) for key in modes+traveltime_mean_modes+dist_mean_modes)#+cellsN_mean_modes)
modesAggSum = dict((key, ['sum']) for key in modes+traveltime_mean_modes+dist_mean_modes)#+cellsN_mean_modes)
modesAggMixed = dict((key, ['sum']) for key in modes)
modesAggMixed1 = dict((key, ['mean']) for key in traveltime_mean_modes+dist_mean_modes)
modesAggMixed.update(modesAggMixed1)

hours = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
ticks = hours.copy()
for i in range(0,24):
    ticks[i] = hours[i]*(360/24)*pi/180

# for horizontal plotting:
def moveMidnight2List(lst):
    lst.append(lst.pop(0))
    lst.append(lst.pop(0))
    return lst;
hours2 = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 1, 2]
ticksPos = [-.5,.5,1.5,2.5,3.5,4.5,5.5,6.5,7.5,8.5,9.5,10.5,11.5,12.5,13.5,14.5,15.5,16.5,17.5,18.5,19.5,20.5,21.5,22.5,23.5]

weekangles = [0]*7
for i in range(0,7):
    weekangles[i] = (i*360/7)*pi/180

## prepare plotdf
function to prepare plotdf and choose between circular and horizontal

In [6]:
def plotDay(modesPlt='', day='', dayType='',
            traveltime=(0,60), distance=(0,60), # check maxDist to set limits
            StartCells='', EndCells='', circular=True):
    
    if(StartCells): startCells = list(map(int, StartCells.split(',')))
    if(EndCells): endCells = list(map(int, EndCells.split(',')))
    
    # aggregate cycledf according to filter choices for day and dayType, mean over days and dayTypes for each flowID and direction
    d.update(modesAggMean)
    if (day):
        plotdf = ODdf[(ODdf.weekday == day)].groupby(['startCellID', 'endCellID', 'hour']).agg(d).copy().reset_index()
        plotdf.columns = plotdf.columns.get_level_values(0)
    elif (dayType):
        plotdf = ODdf[(ODdf.dayType == dayType)].groupby(['startCellID', 'endCellID', 'hour']).agg(d).copy().reset_index()
        plotdf.columns = plotdf.columns.get_level_values(0)
    else:
        plotdf = ODdf.groupby(['startCellID', 'endCellID', 'hour']).agg(d).copy().reset_index()
        plotdf.columns = plotdf.columns.get_level_values(0)
    
    # apply startCell and endCell filters, sum (mean) while aggregating
    d.update(modesAggMixed)
    if(StartCells):
        if(EndCells):
            plotdf = plotdf[(plotdf.startCellID.isin(startCells))&
                          (plotdf.endCellID.isin(endCells))].groupby(['startCellID','endCellID', 'hour']).agg(d).copy().reset_index()
            plotdf.columns = plotdf.columns.get_level_values(0)
        else:
            plotdf = plotdf[(plotdf.startCellID.isin(startCells))#&(~plotdf.endCellID.isin(startCells))
                          ].groupby(['startCellID','endCellID', 'hour']).agg(d).copy().reset_index()
            plotdf.columns = plotdf.columns.get_level_values(0)
    elif(EndCells):
        plotdf = plotdf[(plotdf.endCellID.isin(endCells))#&(~plotdf.startCellID.isin(endCells))
                     ].groupby(['startCellID','endCellID', 'hour']).agg(d).copy().reset_index()
        plotdf.columns = plotdf.columns.get_level_values(0)
    
    # apply traveltime and distance filters
    if(modesPlt):
        ttmode = traveltime_mean_modes[modes.index(modesPlt)]
        distmode = dist_mean_modes[modes.index(modesPlt)]
        plotdf = plotdf[(plotdf[ttmode].between(traveltime[0]*60,traveltime[1]*60,inclusive=True))&
                        (plotdf[distmode].between(distance[0],distance[1],inclusive=True))]
    else:
        plotdf = plotdf[(plotdf.traveltime_mean_moves.between(traveltime[0]*60,traveltime[1]*60,inclusive=True))&
                        (plotdf.dist_mean_moves.between(distance[0],distance[1],inclusive=True))]
        
    # sum (mean) over start and end cells
    plotdf = plotdf.groupby(['hour']).agg(d).copy().reset_index()
    plotdf.columns = plotdf.columns.get_level_values(0)

    if(circular):
        plotDayCircular(plotdf, modesPlt)
    else:
        plotDayHorizontal(plotdf, modesPlt)
    return;

## plotting function for circular plot

In [7]:
def plotDayCircular(plotdf, modesPlt):
    # initialize plot
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111, projection='polar')
    ax.set_theta_direction(-1)
    ax.set_theta_offset(-pi)
    ax.set_rlabel_position(90)
    ax.yaxis.grid(linestyle = (0,(1,5)))
    ax.spines['polar'].set_visible(False)
    
    angles = list(plotdf.hour*(360/24)*pi/180+(pi/24))
    categories=list(plotdf.hour)
    plt.xticks(ticks, hours, color='grey', size=8)
    
    width=pi/13
    alpha=1 
    
    values = []
    if (modesPlt == 'privat'):
        values = list(plotdf.privat)
        maxValues = max(values) if values else 0
        base = .6*maxValues
    elif (modesPlt == 'public'):
        values0 = [0]*24
    else:
        values0 = list(plotdf.privat)
    
    if(values): # only plot privates and return
        ax.bar(angles, values, width=width, alpha=alpha, color=modesColor['privat'])   
        ax.add_artist(plt.Circle((0, 0), maxValues+base, transform=ax.transData._b,
                             fill=False, edgecolor='#777777', linewidth=1, alpha=1, zorder=10))
        ax.set_yticks([maxValues, .75*maxValues, .5*maxValues, .25*maxValues,0])
        if (maxValues>1000):
            ax.set_yticklabels([str(np.round(maxValues/1000,decimals=1))+' Tsd','','',''], color='#777777')
        else:
            ax.set_yticklabels([str(np.int(maxValues)),'','',''], color='#777777')
        ax.set_rmin(-base)
        return ax;
    
    else: # plot stacked bars, then return    
        values1 = list(plotdf.Rail)
        values2 = list(plotdf.Bus)
        values3 = list(plotdf.UBahn)
        values4 = list(plotdf.Tram)
        values01 = np.add(values0, values1).tolist()
        values012 = np.add(values01, values2).tolist()
        values0123 = np.add(values012, values3).tolist()
        values01234 = np.add(values0123, values4).tolist()

        maxAll = max(values01234) if values01234 else 0
        base = .6*maxAll

        ax.bar(angles, values0, width=width, alpha=alpha, color=modesColor['privat'])   
        ax.bar(angles, values1, width=width, alpha=alpha, color=modesColor['Rail'], bottom = values0)
        ax.bar(angles, values2, width=width, alpha=alpha, color=modesColor['Bus'], bottom = values01)
        ax.bar(angles, values3, width=width, alpha=alpha, color=modesColor['UBahn'], bottom = values012)
        ax.bar(angles, values4, width=width, alpha=alpha, color=modesColor['Tram'], bottom = values0123)

        ax.add_artist(plt.Circle((0, 0), maxAll+base, transform=ax.transData._b,
                             fill=False, edgecolor=modesColor['Tram'], linewidth=1, alpha=1, zorder=10))
        ax.set_yticks([maxAll, .75*maxAll, .5*maxAll, .25*maxAll, ])
        if (maxAll>1000):
            ax.set_yticklabels([str(np.round(maxAll/1000,decimals=1))+' Tsd','','',''], color=modesColor['Tram'])
        else:
            ax.set_yticklabels([str(np.int(maxAll)),'','',''], color=modesColor['Tram'])
        ax.set_rmin(-base)
        return ax;

## plotting function for horizontal plot

In [8]:
def plotDayHorizontal(plotdf, modesPlt):
    # initialize plot
    fig = plt.figure(figsize=(12, 5))
    ax = fig.add_subplot(111)
    plt.yticks(color='#777777')
    ax.yaxis.grid(linestyle = (0,(1,5)))
    for spine in ['left','right','top']:
        ax.spines[spine].set_visible(False)
    ax.spines['bottom'].set_position('zero')
    
    ticks = list(plotdf.hour)
    # ticksPos = (list(plotdf.hour-.5)+[23.5])
    plt.xticks(ticksPos, hours2, color='#777777', size=8)
    plt.tick_params(axis='both', which='both', length=0)
    
    width=0.9
    alpha=1 
    
    values = []
    if (modesPlt == 'privat'):
        values = moveMidnight2List(list(plotdf.privat))
        # meanValues = np.mean(values) if values else 0
        maxValues = max(values) if values else 0
    elif (modesPlt == 'public'):
        negticks = []
        negticklabels = []
    else: # plot private values
        values0 = moveMidnight2List(list(-plotdf.privat))
        maxPriv = min(values0) if values0 else 0
        base = .08*maxPriv
        plt.axhline(y=maxPriv+base, color=modesColor['privat'], linewidth=1)
        ax.bar(ticks, values0, width=width, alpha=alpha, color=modesColor['privat'], bottom = base)
        negticks = [base, base+.25*maxPriv, base+.5*maxPriv, base+.75*maxPriv, base+maxPriv]
        if (maxPriv<-1000):
            negticklabels = [0,'','','',str(np.round(-maxPriv/1000, decimals=1))+' Tsd']
        else:
            negticklabels = [0,'','','',str(np.int(-maxPriv))]
        ax.spines['bottom'].set_visible(False)
    
    if(values): # only plot privates and return
        ax.bar(ticks, values, width=width, alpha=alpha, color=modesColor['privat'])
        ax.set_yticks([maxValues, .75*maxValues, .5*maxValues, .25*maxValues, 0])
        if (maxValues>1000):
            ax.set_yticklabels([str(np.round(maxValues/1000,decimals=1))+' Tsd','','','',0], color='#777777', size=8)
        else:
            ax.set_yticklabels([str(np.int(maxValues)),'','','',0], color='#777777', size=8)
        plt.axhline(y=maxValues, color=modesColor['privat'], linewidth=1)
        ax.spines['bottom'].set_linestyle((0,(1,5)))
        ax.spines['bottom'].set_color('#777777')
        return ax;
    
    else: # plot public values and return    
        values1 = moveMidnight2List(list(plotdf.Rail))
        values2 = moveMidnight2List(list(plotdf.Bus))
        values3 = moveMidnight2List(list(plotdf.UBahn))
        values4 = moveMidnight2List(list(plotdf.Tram))
        values12 = np.add(values1, values2).tolist()
        values123 = np.add(values12, values3).tolist()
        values1234 = np.add(values123, values4).tolist()

        maxAll = max(values1234) if values1234 else 0
        # meanAll = np.mean(values01234) if values01234 else 0
  
        ax.bar(ticks, values1, width=width, alpha=alpha, color=modesColor['Rail'])
        ax.bar(ticks, values2, width=width, alpha=alpha, color=modesColor['Bus'], bottom = values1)
        ax.bar(ticks, values3, width=width, alpha=alpha, color=modesColor['UBahn'], bottom = values12)
        ax.bar(ticks, values4, width=width, alpha=alpha, color=modesColor['Tram'], bottom = values123)
        plt.axhline(y=maxAll, color=modesColor['Tram'], linewidth=1)
        
        ax.set_yticks([maxAll, .75*maxAll, .5*maxAll, .25*maxAll, 0]+negticks)
        if (maxAll>1000):
            ax.set_yticklabels([str(np.round(maxAll/1000,decimals=1))+' Tsd','','','',0]+negticklabels, color=modesColor['Tram'], size=8)
        else:
            ax.set_yticklabels([str(np.int(maxAll)),'','','',0]+negticklabels, color=modesColor['Tram'], size=8)
        ax.spines['bottom'].set_linestyle((0,(1,5)))
        ax.spines['bottom'].set_color('#777777')
        return ax;

## prepare range sliders etc

In [9]:
range_slider_h = pyw.IntRangeSlider(
    value=[18,24],
    min=0, max=24, step=1,
    description='Time of day')

range_slider_tt = pyw.IntRangeSlider(
    value=[0,60],
    min=0, max=60, step=1,
    description='Minutes travelled')

range_slider_td = pyw.IntRangeSlider(
    value=[0,60],
    min=0, max=60, step=1,
    description='km travelled')

days = ['', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dayTypes = ['', 'MonThu', 'Fri', 'Weekend', 'Weekend']

## plot data

In [10]:
pyw.interact_manual(plotDay,
                    modesPlt = modesPlt,
                    StartCells = '',
                    EndCells = '',
                    day = days, dayType = dayTypes,
                    hours = range_slider_h,
                    traveltime = range_slider_tt,
                    distance = range_slider_td)

interactive(children=(Dropdown(description='modesPlt', options=('', 'privat', 'public'), value=''), Dropdown(d…

<function __main__.plotDay(modesPlt='', day='', dayType='', traveltime=(0, 60), distance=(0, 60), StartCells='', EndCells='', circular=True)>